In [32]:
!pip install keras

In [33]:
!pip install tensorflow --user

In [34]:
import pandas as pd
import numpy as np
import keras as kr
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.optimizers import Adam

In [35]:
# Load data from CSV
df = pd.read_csv('malicious_phish - Copy.csv', on_bad_lines='skip')

In [36]:
# Extract URLs and labels
urls = df['url'].tolist()
labels = df['type'].tolist()

# Tokenise and preprocess the URLs
tokeniser = Tokenizer()
tokeniser.fit_on_texts(urls)
sequences = tokeniser.texts_to_sequences(urls)

# Padding sequences to have the same length
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Convert labels to numpy array
labels = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

#callback = kr.callbacks.EarlyStopping(monitor='accuracy')

# Model definition
model = Sequential()#Sequential model
model.add(Embedding(input_dim=len(tokeniser.word_index) + 1, output_dim=32, input_length=max_length))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid')) #activation function

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.1), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=2500, validation_split=0.1)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

Epoch 1/10
188/188 [==============================] - 357s 2s/step - loss: 0.6326 - accuracy: 0.6664 - val_loss: 0.2287 - val_accuracy: 0.9254
Epoch 2/10
188/188 [==============================] - 352s 2s/step - loss: 0.1844 - accuracy: 0.9339 - val_loss: 0.1610 - val_accuracy: 0.9395
Epoch 3/10
188/188 [==============================] - 348s 2s/step - loss: 0.1115 - accuracy: 0.9619 - val_loss: 0.1585 - val_accuracy: 0.9419
Epoch 4/10
188/188 [==============================] - 346s 2s/step - loss: 0.0950 - accuracy: 0.9682 - val_loss: 0.1556 - val_accuracy: 0.9438
Epoch 5/10
188/188 [==============================] - 347s 2s/step - loss: 0.0871 - accuracy: 0.9708 - val_loss: 0.1562 - val_accuracy: 0.9445
Epoch 6/10
188/188 [==============================] - 344s 2s/step - loss: 0.0793 - accuracy: 0.9730 - val_loss: 0.1587 - val_accuracy: 0.9437
Epoch 7/10
188/188 [==============================] - 346s 2s/step - loss: 0.0761 - accuracy: 0.9739 - val_loss: 0.1653 - val_accuracy: 0.9455

In [39]:
# Make predictions on new data
new_urls = [input()]
new_sequences = tokeniser.texts_to_sequences(new_urls)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_length, padding='post')
predictions = model.predict(new_padded_sequences)

# Print the predictions
for url, prediction in zip(new_urls, predictions):
    print(f"{url} - Predicted Probability: {prediction[0]}")

facebook.com
1/1 [==============================] - 0s 23ms/step
facebook.com - Predicted Probability: 0.003370010992512107
